## Pipeline Orchestrations Tutorial

Wallaroo provides data connections, orchestrations, and tasks to provide organizations with a method of creating and managing automated tasks that can either be run on demand, on a regular schedule, or as a service so they respond to requests.

| Object | Description |
|---|---|
| Orchestration | A set of instructions written as a python script with a requirements library.  Orchestrations are uploaded to the Wallaroo instance |
| Task | An implementation of an orchestration.  Tasks are run either once when requested, on a repeating schedule, or as a service. |
| Connection | Definitions set by MLOps engineers that are used by other Wallaroo users for connection information to a data source.  Usually paired with orchestrations. |

A typical flow in the orchestration, task and connection life cycle is:

1. (Optional) A connection is defined with information such as username, connection URL, tokens, etc.
1. One or more connections are applied to a workspace for users to implement in their code or orchestrations.
1. An orchestration is created to perform some set instructions.  For example:
    1. Deploy a pipeline, request data from an external service, store the results in an external database, then undeploy the pipeline.
    1. Download a ML Model then replace a current pipeline step with the new version.
    1. Collect log files from a deployed pipeline once every hour and submit it to a Kafka or other service.
1. A task is created that specifies the orchestration to perform and the schedule:
    1. Run once.
    1. Run on a schedule (based on `cron` like settings).
    1. Run as a service to be run whenever requested.
1. Once the use for a task is complete, it is killed and its schedule or service removed.

## Tutorial Goals

The tutorial will demonstrate the following:

1. Create a simple connection to retrieve an Apache Arrow table file from a GitHub registry.
1. Create an orchestration that retrieves the Apache Arrow table file from the location defined by the connection, deploy a pipeline, perform an inference, then undeploys the pipeline.
1. Implement the orchestration as a task that runs every minute.
1. Display the logs from the pipeline after 5 minutes to verify the task is running.

## Required Libraries

The following libraries are required for this tutorial, and included by default in a Wallaroo instance's JupyterHub service.

* [wallaroo](https://pypi.org/project/wallaroo/):  The Wallaroo SDK.
* [pandas](https://pypi.org/project/pandas/): The pandas data analysis library.
* [pyarrow](https://pypi.org/project/pyarrow/): The Apache Arrow Python library.

The specific versions used are set in the file `./resources/requirements.txt`.  Supported libraries are automatically installed with the `pypi` or `conda` commands.  For example, from the root of this tutorials folder:

```python
pip install -r ./resources/requirements.txt
```



Using pipeline orchestrations consist of these steps: 
1. [Write orchestration code](#1.-Write-orchestration-code)
2. [Create archive](#2.-Create-archive)
3. [Upload archive](#3.-Upload-archive)
4. [Run task](#4.-Run-task)

[Task Management](#Task-Management)

## Connect to the Wallaroo Instance

The first step is to connect to a Wallaroo instance.  We'll load the libraries and set our client connection settings

### Workspace, Model and Pipeline Setup

For this tutorial, we'll create a workspace, upload our sample model and deploy a pipeline.  We'll perform some quick sample inferences to verify that everything it working.

In [2]:
import wallaroo
from wallaroo.object import EntityNotFoundError

# to display dataframe tables
from IPython.display import display
# used to display dataframe information without truncating
import pandas as pd
pd.set_option('display.max_colwidth', None)
import pyarrow as pa

import os
# Used for the Wallaroo SDK version 2023.1
os.environ["ARROW_ENABLED"]="True"

import requests

In [49]:
# Login through local Wallaroo instance

# wl = wallaroo.Client()

# # SSO login through keycloak

# wallarooPrefix = "YOUR PREFIX"
# wallarooSuffix = "YOUR PREFIX"


# wallarooPrefix = "doc-test"
# wallarooSuffix = "wallaroocommunity.ninja"


# wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
#                     auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
#                     auth_type="sso")

# os.environ["WALLAROO_SDK_CREDENTIALS"] = './creds.json.example'

wallarooPrefix="doc-test"
wallarooSuffix="wallaroocommunity.ninja"

# wallarooPrefix="product-uat-ee"
# wallarooSuffix="wallaroocommunity.ninja"

# wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
#                     auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
#                     auth_type="sso")

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
                    auth_type="sso")

Please log into the following URL in a web browser:

	https://doc-test.keycloak.wallaroocommunity.ninja/auth/realms/master/device?user_code=BRBM-QMTL

Login successful!


In [4]:
# Setting variables for later steps

workspace_name = 'orchestrationworkspace'
pipeline_name = 'orchestrationpipeline'
model_name = 'orchestrationmodel'
model_file_name = './models/rf_model.onnx'
connection_name = "houseprice_arrow_table"

### Helper Methods

The following helper methods are used to either create or get workspaces and pipelines.

In [5]:
# helper methods to retrieve workspaces and pipelines

def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

### Create the Workspace and Pipeline

We'll now create our workspace and pipeline for the tutorial.  If this tutorial has been run previously, then this will retrieve the existing ones with the assumption they're for us with this tutorial.

We'll set the retrieved workspace as the current workspace in the SDK, so all commands will default to that workspace.

In [6]:
workspace = get_workspace(workspace_name)
wl.set_current_workspace(workspace)

pipeline = get_pipeline(pipeline_name)

### Upload the Model and Deploy Pipeline

We'll upload our model into our sample workspace, then add it as a pipeline step before deploying the pipeline to it's ready to accept inference requests.

In [15]:
# Upload the model

housing_model_control = wl.upload_model(model_name, model_file_name).configure()

# Add the model as a pipeline step

pipeline.add_model_step(housing_model_control)

name,orchestrationpipeline
created,2023-05-04 17:44:59.092580+00:00
last_updated,2023-05-04 17:45:03.098842+00:00
deployed,False
tags,
versions,"b9693fe6-3838-410c-aaf2-5ede4adb3a02, b01b49f3-f840-4366-b073-20644d1f6c12"
steps,orchestrationmodel


In [16]:
#deploy the pipeline
pipeline.deploy()

name,orchestrationpipeline
created,2023-05-04 17:44:59.092580+00:00
last_updated,2023-05-04 17:50:56.665499+00:00
deployed,True
tags,
versions,"8630f633-1b71-4805-a09f-62daf83a101b, b9693fe6-3838-410c-aaf2-5ede4adb3a02, b01b49f3-f840-4366-b073-20644d1f6c12"
steps,orchestrationmodel


### Sample Inferences

We'll perform some quick sample inferences using an Apache Arrow table as the input.  Once that's finished, we'll undeploy the pipeline and return the resources back to the Wallaroo instance.

In [17]:
# sample inferences

batch_inferences = pipeline.infer_from_file('./data/xtest-1k.arrow')

large_inference_result =  batch_inferences.to_pandas()
display(large_inference_result.head(20))

,time,in.tensor,out.variable,check_failures
0,2023-05-04 17:51:08.515,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]",[718013.75],0
1,2023-05-04 17:51:08.515,"[2.0, 2.5, 2170.0, 6361.0, 1.0, 0.0, 2.0, 3.0, 8.0, 2170.0, 0.0, 47.7109, -122.017, 2310.0, 7419.0, 6.0, 0.0, 0.0]",[615094.56],0
2,2023-05-04 17:51:08.515,"[3.0, 2.5, 1300.0, 812.0, 2.0, 0.0, 0.0, 3.0, 8.0, 880.0, 420.0, 47.5893, -122.317, 1300.0, 824.0, 6.0, 0.0, 0.0]",[448627.72],0
3,2023-05-04 17:51:08.515,"[4.0, 2.5, 2500.0, 8540.0, 2.0, 0.0, 0.0, 3.0, 9.0, 2500.0, 0.0, 47.5759, -121.994, 2560.0, 8475.0, 24.0, 0.0, 0.0]",[758714.2],0
4,2023-05-04 17:51:08.515,"[3.0, 1.75, 2200.0, 11520.0, 1.0, 0.0, 0.0, 4.0, 7.0, 2200.0, 0.0, 47.7659, -122.341, 1690.0, 8038.0, 62.0, 0.0, 0.0]",[513264.7],0
5,2023-05-04 17:51:08.515,"[3.0, 2.0, 2140.0, 4923.0, 1.0, 0.0, 0.0, 4.0, 8.0, 1070.0, 1070.0, 47.6902, -122.339, 1470.0, 4923.0, 86.0, 0.0, 0.0]",[668288.0],0
6,2023-05-04 17:51:08.515,"[4.0, 3.5, 3590.0, 5334.0, 2.0, 0.0, 2.0, 3.0, 9.0, 3140.0, 450.0, 47.6763, -122.267, 2100.0, 6250.0, 9.0, 0.0, 0.0]",[1004846.5],0
7,2023-05-04 17:51:08.515,"[3.0, 2.0, 1280.0, 960.0, 2.0, 0.0, 0.0, 3.0, 9.0, 1040.0, 240.0, 47.602, -122.311, 1280.0, 1173.0, 0.0, 0.0, 0.0]",[684577.2],0
8,2023-05-04 17:51:08.515,"[4.0, 2.5, 2820.0, 15000.0, 2.0, 0.0, 0.0, 4.0, 9.0, 2820.0, 0.0, 47.7255, -122.101, 2440.0, 15000.0, 29.0, 0.0, 0.0]",[727898.1],0
9,2023-05-04 17:51:08.515,"[3.0, 2.25, 1790.0, 11393.0, 1.0, 0.0, 0.0, 3.0, 8.0, 1790.0, 0.0, 47.6297, -122.099, 2290.0, 11894.0, 36.0, 0.0, 0.0]",[559631.1],0


In [18]:
# undeploy the pipeline

pipeline.undeploy()

name,orchestrationpipeline
created,2023-05-04 17:44:59.092580+00:00
last_updated,2023-05-04 17:50:56.665499+00:00
deployed,False
tags,
versions,"8630f633-1b71-4805-a09f-62daf83a101b, b9693fe6-3838-410c-aaf2-5ede4adb3a02, b01b49f3-f840-4366-b073-20644d1f6c12"
steps,orchestrationmodel


## Create Wallaroo Connection




Connections are created at the Wallaroo instance level, typically by a MLOps or DevOps engineer, then applied to a workspace.

For this section:

1. We will create a sample connection that just has a URL to the same Arrow table file we used in the previous step.
1. We'll apply the data connection to the workspace above.
1. For a quick demonstration, we'll use the connection to retrieve the Arrow table file and use it for a quick sample inference.

### Create Connection

Connections are created with the Wallaroo client command [`create_connection`](https://staging.docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-dataconnectors/#create-data-connection) with the following parameters.

| Parameter | Type | Description |
| --- | --- | ---|
| **name** | string (Required) | The name of the connection. |
| **type** | string (Required) | The user defined type of connection. |
| **details** | Dict (Requires) | User defined configuration details for the data connection.  These can be `{'username':'dataperson', 'password':'datapassword', 'port': 3339}`, or `{'token':'abcde123==', 'host':'example.com', 'port:1234'}`, or other user defined combinations.  |

We'll create the connection named `houseprice_arrow_table`, set it to the type `HTTPFILE`, and provide the details as `'host':'https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/20230314_2023.2_updates/wallaroo-testing-tutorials/houseprice-saga/data/xtest-1k.arrow?raw=true'` - the location for our sample Arrow table inference input.

In [7]:
wl.create_connection(connection_name, 
                  "HTTPFILE", 
                  {'host':'https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/20230314_2023.2_updates/wallaroo-testing-tutorials/houseprice-saga/data/xtest-1k.arrow?raw=true'}
                  )

Field,Value
Name,houseprice_arrow_table
Connection Type,HTTPFILE
Details,*****
Created At,2023-05-04T21:34:53.380085+00:00


### List Data Connections

The Wallaroo Client [`list_connections()`](https://staging.docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-dataconnectors/#list-data-connections) method lists all connections for the Wallaroo instance.


In [20]:
wl.list_connections()

name,connection type,details,created at
houseprice_arrow_table,HTTPFILE,*****,2023-05-04T17:52:32.249322+00:00


### Add Connection to Workspace

The method Workspace [`add_connection(connection_name)`](https://staging.docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-dataconnectors/#add-data-connection-to-workspace) adds a Data Connection to a workspace, and takes the following parameters.

| Parameter | Type | Description |
| --- | --- | ---|
| **name** | string (Required) | The name of the Data Connection |

We'll add this connection to our sample workspace.

In [8]:
workspace.add_connection(connection_name)

### List Connections in a Workspace

The method Workspace `list_connections()` displays a list of connections attached to the workspace.  By default the `details` field is obfuscated.  We'll list the connections in our sample workspace, then use that to retrieve our specific connection.

In [9]:
connections = workspace.list_connections()
display(connections)


name,connection type,details,created at
houseprice_arrow_table,HTTPFILE,*****,2023-05-04T21:34:53.380085+00:00


In [10]:
connection = connections[0]
display(connection)

Field,Value
Name,houseprice_arrow_table
Connection Type,HTTPFILE
Details,*****
Created At,2023-05-04T21:34:53.380085+00:00


### Connection Details

The Connection method `details()` retrieves a the connection `details()` as a `dict`.

In [50]:
display(connection.details())

{'host': 'https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/20230314_2023.2_updates/wallaroo-testing-tutorials/houseprice-saga/data/xtest-1k.arrow?raw=true'}

### Using a Connection Example

For this example, the connection will be used to retrieve the Apache Arrow file referenced in the connection, and use that to turn it into an Apache Arrow table, then use that for a sample inference.

In [11]:
# Deploy the pipeline 
pipeline.deploy()

# Retrieve the file
# set accept as apache arrow table
headers = {
    'Accept': 'application/vnd.apache.arrow.file'
}

response = requests.get(
                    connection.details()['host'], 
                    headers=headers
                )

# Arrow table is retrieved 
with pa.ipc.open_file(response.content) as reader:
    arrow_table = reader.read_all()

results = pipeline.infer(arrow_table)

result_table = results.to_pandas()
display(result_table.head(20))

# Undeploy the pipeline and return the resources back to the Wallaroo instance
pipeline.undeploy()

,time,in.tensor,out.variable,check_failures
0,2023-05-04 21:35:40.019,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]",[718013.75],0
1,2023-05-04 21:35:40.019,"[2.0, 2.5, 2170.0, 6361.0, 1.0, 0.0, 2.0, 3.0, 8.0, 2170.0, 0.0, 47.7109, -122.017, 2310.0, 7419.0, 6.0, 0.0, 0.0]",[615094.56],0
2,2023-05-04 21:35:40.019,"[3.0, 2.5, 1300.0, 812.0, 2.0, 0.0, 0.0, 3.0, 8.0, 880.0, 420.0, 47.5893, -122.317, 1300.0, 824.0, 6.0, 0.0, 0.0]",[448627.72],0
3,2023-05-04 21:35:40.019,"[4.0, 2.5, 2500.0, 8540.0, 2.0, 0.0, 0.0, 3.0, 9.0, 2500.0, 0.0, 47.5759, -121.994, 2560.0, 8475.0, 24.0, 0.0, 0.0]",[758714.2],0
4,2023-05-04 21:35:40.019,"[3.0, 1.75, 2200.0, 11520.0, 1.0, 0.0, 0.0, 4.0, 7.0, 2200.0, 0.0, 47.7659, -122.341, 1690.0, 8038.0, 62.0, 0.0, 0.0]",[513264.7],0
5,2023-05-04 21:35:40.019,"[3.0, 2.0, 2140.0, 4923.0, 1.0, 0.0, 0.0, 4.0, 8.0, 1070.0, 1070.0, 47.6902, -122.339, 1470.0, 4923.0, 86.0, 0.0, 0.0]",[668288.0],0
6,2023-05-04 21:35:40.019,"[4.0, 3.5, 3590.0, 5334.0, 2.0, 0.0, 2.0, 3.0, 9.0, 3140.0, 450.0, 47.6763, -122.267, 2100.0, 6250.0, 9.0, 0.0, 0.0]",[1004846.5],0
7,2023-05-04 21:35:40.019,"[3.0, 2.0, 1280.0, 960.0, 2.0, 0.0, 0.0, 3.0, 9.0, 1040.0, 240.0, 47.602, -122.311, 1280.0, 1173.0, 0.0, 0.0, 0.0]",[684577.2],0
8,2023-05-04 21:35:40.019,"[4.0, 2.5, 2820.0, 15000.0, 2.0, 0.0, 0.0, 4.0, 9.0, 2820.0, 0.0, 47.7255, -122.101, 2440.0, 15000.0, 29.0, 0.0, 0.0]",[727898.1],0
9,2023-05-04 21:35:40.019,"[3.0, 2.25, 1790.0, 11393.0, 1.0, 0.0, 0.0, 3.0, 8.0, 1790.0, 0.0, 47.6297, -122.099, 2290.0, 11894.0, 36.0, 0.0, 0.0]",[559631.1],0


name,orchestrationpipeline
created,2023-05-04 17:44:59.092580+00:00
last_updated,2023-05-04 21:35:28.752580+00:00
deployed,False
tags,
versions,"83a37dea-d49a-4f73-a00d-f40d06de5ec8, e18cd76d-24b6-4f39-8383-22744b420af6, 4c7082a3-0b10-4251-bf29-17d62b8d38b2, 8bd83a24-6743-43a2-b1f3-7fe0866d7f14, 7af43112-b3c0-4b4c-94d6-8211229b1846, 1ea9a21e-d1c2-4ae8-9f5d-32e43a03e1e9, 5796cd66-127b-45a8-898a-0541c7dc0e43, 8630f633-1b71-4805-a09f-62daf83a101b, b9693fe6-3838-410c-aaf2-5ede4adb3a02, b01b49f3-f840-4366-b073-20644d1f6c12"
steps,orchestrationmodel


### Remove Connection from Workspace

The Workspace method `remove_connection(connection_name)` removes the connection from the workspace, but does not delete the connection from the Wallaroo instance.  This method takes the following parameters.

| Parameter | Type | Description |
|---|---|---|
| **name** | String (Required) | The name of the connection to be removed 

The previous connection will be removed from the workspace, then the workspace connections displayed to verify it has been removed.

In [12]:
workspace.remove_connection(connection_name)

display(workspace.list_connections())

[]

### Delete Connection

The Connection method `delete_connection()` removes the connection from the Wallaroo instance, and all attachments in workspaces they were connected to.

In [13]:
connection.delete_connection()

wl.list_connections()

[]

## Orchestration Tutorial

The next series of examples will build on what we just did.  So far we have:

* Deployed a pipeline, performed sample inferences with a local Apache Arrow file, displayed the results, then undeployed the pipeline.
* Deployed a pipeline, use a Wallaroo connection details to retrieve a remote Apache Arrow file, performed inferences and displayed the results, then undeployed the pipeline.

For the orchestration tutorial, we'll do the same thing only package it into a separate python script and upload it to the Wallaroo instance, then create a task from that orchestration and perform our sample inferences again.

### Orchestration Requirements

Orchestrations are uploaded to the Wallaroo instance as a ZIP file with the following requirements:

* The ZIP file should not contain any directories - only files at the top level.

| Parameter | Type | Description |
|---|---|---|
| **User Code** | (**Required**) Python script as `.py` files | Python scripts for the orchestration to run.  If the file `main.py` exists, that will be the entrypoint.  Otherwise, if only one .py exists, then that will be the entrypoint. |
| Python Library Requirements | (**Required**) `requirements.txt` file in the [requirements file format](https://pip.pypa.io/en/stable/reference/requirements-file-format/).  This is in the root of the zip file, and there can only be one `requirements.txt` file for the orchestration. |
| Other artifacts | &nbsp; | Other artifacts such as files, data, or code to support the orchestration.

#### Zip Instructions

In a terminal with the `zip` command, assemble artifacts as above and then create the archive.  The `zip` command is included by default with the Wallaroo JupyterHub service.

`zip` commands take the following format, with `{zipfilename}.zip` as the zip file to save the artifacts to, and each file thereafter as the files to add to the archive.

```bash
zip {zipfilename}.zip file1, file2, file3....
```

For example, the following command will add the files `main.py` and `requirements.txt` into the file `hello.zip`.

```shell
$ zip hello.zip main.py requirements.txt 
  adding: main.py (deflated 47%)
  adding: requirements.txt (deflated 52%)
```

### Orchestration Recommendations

The following recommendations will make using Wallaroo orchestrations 

* The version of Python used should match the same version as in the Wallaroo JupyterHub service.
* The same version of the [Wallaroo SDK](https://pypi.org/project/wallaroo/) should match the server.  For a 2023.2 Wallaroo instance, use the Wallaroo SDK version 2023.2.
* Specify the version of `pip` dependencies.
* The `wallaroo.Client` constructor `auth_type` argument is ignored.  Using `wallaroo.Client()` is sufficient.
* The following methods will assist with orchestrations:
  * `wallaroo.in_task()` :  Returns `True` if the code is running within an Orchestrator task.
  * `wallaroo.task_args()`:  Returns a `Dict` of invocation-specific arguments passed to the `run_` calls.

### Example requirements.txt file

```python
dbt-bigquery==1.4.3
dbt-core==1.4.5
dbt-extractor==0.4.1
dbt-postgres==1.4.5
google-api-core==2.8.2
google-auth==2.11.0
google-auth-oauthlib==0.4.6
google-cloud-bigquery==3.3.2
google-cloud-bigquery-storage==2.15.0
google-cloud-core==2.3.2
google-cloud-storage==2.5.0
google-crc32c==1.5.0
google-pasta==0.2.0
google-resumable-media==2.3.3
googleapis-common-protos==1.56.4
```

### Sample Orchestrator

The following orchestrator artifacts are in the directory `./remote_inference` and includes the file `main.py` with the following code:

```python
import wallaroo
from wallaroo.object import EntityNotFoundError
import pandas as pd
import pyarrow as pa
import requests

wl = wallaroo.Client()

# Setting variables for later steps

workspace_name = 'orchestrationworkspace'
pipeline_name = 'orchestrationpipeline'
model_name = 'orchestrationmodel'
model_file_name = './models/rf_model.onnx'
connection_name = "houseprice_arrow_table"

# helper methods to retrieve workspaces and pipelines

def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

workspace = get_workspace(workspace_name)
wl.set_current_workspace(workspace)

pipeline = get_pipeline(pipeline_name)

#deploy the pipeline
pipeline.deploy()

# Get the connection - assuming it will be the only one

connection = workspace.list_connections()[0]

# Deploy the pipeline 
pipeline.deploy()

# Retrieve the file
# set accept as apache arrow table
headers = {
    'Accept': 'application/vnd.apache.arrow.file'
}

response = requests.get(
                    connection.details()['host'], 
                    headers=headers
                )

# Arrow table is retrieved 
with pa.ipc.open_file(response.content) as reader:
    arrow_table = reader.read_all()

# Perform the inference
results = pipeline.infer(arrow_table)

# Undeploy the pipeline and return the resources back to the Wallaroo instance
pipeline.undeploy()
```

This is saved to the file `./remote_inference/remote_inference.zip`.

### Preparing the Wallaroo Instance

To prepare the Wallaroo instance, we'll once again create the Wallaroo connection `houseprice_arrow_table` and apply it to the workspace.

In [ ]:
wl.create_connection(connection_name, 
                  "HTTPFILE", 
                  {'host':'https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/20230314_2023.2_updates/wallaroo-testing-tutorials/houseprice-saga/data/xtest-1k.arrow?raw=true'}
                  )

workspace.add_connection(connection_name)

### Upload the Orchestration

Orchestrations are uploaded with the Wallaroo client `upload_orchestration(path)` method with the following parameters.

| Parameter | Type | Description |
| --- | --- | ---|
| **path** | string (Required) | The path to the ZIP file to be uploaded. |

Once uploaded, the deployment will be prepared and any requirements will be downloaded and installed.


For this example, the orchestration `./remote_inference/remote_inference.zip` will be uploaded and saved to the variable `orchestration`.

In [33]:
orchestration = wl.upload_orchestration(path="./remote_inference/remote_inference.zip")

### Orchestration Status

The Orchestration method `status()` displays the current status of the uploaded orchestration.

| Status | Description |
| `packaging` | The orchestration is being packaged for use with the Wallaroo instance. |
| `ready` | The orchestration is ready for use. |

In [37]:
orchestration.status()

'ready'

### List Orchestrations

Orchestrations are listed with the Wallaroo Client `list_orchestrations()` which returns a list of available orchestrations.

In [51]:
wl.list_orchestrations()

id,status,name,sha,created at,updated at
ddfdc233-6fa3-4571-bf74-a5c6b1cbddcc,packaging,remote_inference.zip,e8c9da960cb1e05b6619b9fee67798e083f948caa8c84a58df85a785472c5c9e,2023-04-May 21:50:25,2023-04-May 21:50:36
f9cca340-7903-4495-aa39-266e18dec55a,ready,hello.zip,e313b1895af56c83828a31e99dad531400dd5ae3a3bef4b0127606d4fa6556a4,2023-04-May 22:04:44,2023-04-May 22:06:11
c3989f6a-d299-41e9-bf08-a2d04f05ec91,ready,remote_inference.zip,8c9145c27570508d8f2532a040a5a07d34cbb6ed6a8f210a9dcdb73e277fd938,2023-04-May 22:12:41,2023-04-May 22:13:37


## Task Management Tutorial

Once an Orchestration has the status `ready`, it can be run as a task.  Tasks have three run options.

| Type       | SDK Call |  How triggered                                                               | Purpose                                                       |
|------------|----------|:------------------------------------------------------------------------------|:---------------------------------------------------------------|
| Once       | `orchestration.run_once()` | User makes one api call. Task runs once and exits.| Single batch, experimentation                                 |
| Scheduled  | `orchestration.run_scheduled()` | User provides schedule. Task runs exits whenever schedule dictates. | Recurrent batch ETL.|
| Continuous | `orchestration.run_continuously()` | User provides a listen port.  The task listens on that port and is activated when referenced.  @TODO get more details | User defined network service, continuous ETL, queue processor. |

### Task Run Once

Tasks are generated and run once with the Orchestration `run_once(**args)` method.  Any arguments for the orchestration are passed in as a `Dict`.  If there are no arguments, then an empty set `{}` is passed.

In [40]:
# Example: run once
task = orchestration.run_once({})
task

Field,Value
ID,d1c060a4-3beb-473a-b1a9-95de2e658073
Status,pending
Type,Temporary Run
Created At,2023-04-May 22:24:18
Updated At,2023-04-May 22:24:18


### Task Status

The list of tasks in the Wallaroo instance is retrieves through the Wallaroo Client `list_tasks()` method.  This returns an array list of the following.

{{<table "table table-striped table-bordered">}}
| Parameter | Type | Description |
| --- | --- | ---|
| **id** | string | The UUID identifier for the task. |
| **status** | enum | The status of the enum.  Values are: <br><ul><li>`started`: The task has been scheduled to execute.</li><li>`running`: The task is currently executing.</li><li>`pending_kill`: The task kill command has been issued and the task is scheduled to be stopped.</li></ul> |
| **type** | string | The type of the task.  Values are: <br><ul><li>`Temporary Run`: The task runs once then stop.</li><li>`Scheduled Run`: The task repeats on a `cron` like schedule.</li><li>`Service Run`: The task runs as a service and executes when its service port is activated. |
| **created at** | DateTime | The date and time the task was started. |
| **updated at** | DateTime | The date and time the task was updated. |
{{</table>}}

For this example, the status of the previously created task will be generated.

In [65]:
task.status()

'started'

### Kill a Task

Killing a task removes the schedule or removes it from a service.  Tasks are killed with the Task `kill()` method, and returns a message with the status of the kill procedure.

If necessary, all tasks can be killed through the following script.

```python
# Kill all tasks
for t in wl.list_tasks(): t.kill()
```

### Task Results

We can view the inferences from our logs and verify that new entries were added from our task.

In [66]:
pipeline.logs()

,time,in.tensor,out.variable,check_failures
0,2023-05-04 21:35:40.019,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]",[718013.75],0
1,2023-05-04 21:35:40.019,"[2.0, 2.5, 2170.0, 6361.0, 1.0, 0.0, 2.0, 3.0, 8.0, 2170.0, 0.0, 47.7109, -122.017, 2310.0, 7419.0, 6.0, 0.0, 0.0]",[615094.56],0
2,2023-05-04 21:35:40.019,"[3.0, 2.5, 1300.0, 812.0, 2.0, 0.0, 0.0, 3.0, 8.0, 880.0, 420.0, 47.5893, -122.317, 1300.0, 824.0, 6.0, 0.0, 0.0]",[448627.72],0
3,2023-05-04 21:35:40.019,"[4.0, 2.5, 2500.0, 8540.0, 2.0, 0.0, 0.0, 3.0, 9.0, 2500.0, 0.0, 47.5759, -121.994, 2560.0, 8475.0, 24.0, 0.0, 0.0]",[758714.2],0
4,2023-05-04 21:35:40.019,"[3.0, 1.75, 2200.0, 11520.0, 1.0, 0.0, 0.0, 4.0, 7.0, 2200.0, 0.0, 47.7659, -122.341, 1690.0, 8038.0, 62.0, 0.0, 0.0]",[513264.7],0
...,...,...,...,...
512,2023-05-04 21:35:40.019,"[4.0, 2.5, 2740.0, 43101.0, 2.0, 0.0, 0.0, 3.0, 9.0, 2740.0, 0.0, 47.7649, -122.049, 2740.0, 33447.0, 21.0, 0.0, 0.0]",[727898.1],0
513,2023-05-04 21:35:40.019,"[4.0, 3.25, 3320.0, 8587.0, 3.0, 0.0, 0.0, 3.0, 11.0, 2950.0, 370.0, 47.691, -122.337, 1860.0, 5668.0, 6.0, 0.0, 0.0]",[1130661.0],0
514,2023-05-04 21:35:40.019,"[4.0, 2.5, 3130.0, 13202.0, 2.0, 0.0, 0.0, 3.0, 10.0, 3130.0, 0.0, 47.5878, -121.976, 2840.0, 10470.0, 19.0, 0.0, 0.0]",[879083.4],0
515,2023-05-04 21:35:40.019,"[2.0, 1.75, 1370.0, 5125.0, 1.0, 0.0, 0.0, 5.0, 6.0, 1370.0, 0.0, 47.6926, -122.346, 1200.0, 5100.0, 70.0, 0.0, 0.0]",[444933.16],0
